# Creating a feature matrix from a DB Query

In [1]:
import sys
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [2]:
#load in data base
db = pd.read_csv('/Users/Sabatini Lab/GitHub/mouse_bandit/mike_session_record.csv',index_col=0)

### Query all 90-10 sessions where performance > 0.7 block structure was 50 and had no laser trials

In [3]:
r = db[((db['Left Reward Prob'] == 0.9) |  (db['Right Reward Prob'] == 0.9))].copy()
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['Block Range Min'] == 50].copy()
r = r[r['Laser Stim Prob'] == 0].copy()

session_names = r['Session ID'].values 

In [4]:
r.tail(10)

,Owner,Session ID,Mouse ID,Date,Phase,Laser Stim Prob,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
1070,mike,07252017_SOM41,SOM41,2017-07-25 00:00:00,2,0.0,0.1,0.9,50,50,432,6,344,0.89,2,1.0,40,40
1078,mike,07262017_SOM44,SOM44,2017-07-26 00:00:00,2,0.0,0.9,0.1,50,50,301,4,234,0.85,2,1.0,40,40
1079,mike,07262017_SOM45,SOM45,2017-07-26 00:00:00,2,0.0,0.9,0.1,50,50,548,8,433,0.86,2,1.0,40,40
1082,mike,07272017_SOM41,SOM41,2017-07-27 00:00:00,2,0.0,0.1,0.9,50,50,315,5,250,0.86,2,1.0,40,40
1084,mike,07272017_SOM44,SOM44,2017-07-27 00:00:00,2,0.0,0.1,0.9,50,50,222,3,162,0.80,2,1.0,40,40
1085,mike,07272017_SOM45,SOM45,2017-07-27 00:00:00,2,0.0,0.9,0.1,50,50,578,8,443,0.86,2,1.0,40,40
1088,mike,07282017_SOM41,SOM41,2017-07-28 00:00:00,2,0.0,0.9,0.1,50,50,87,1,61,0.74,2,1.0,40,40
1093,mike,07302017_SOM41,SOM41,2017-07-30 00:00:00,2,0.0,0.9,0.1,50,50,545,8,445,0.90,2,1.0,40,40
1095,mike,07302017_SOM45,SOM45,2017-07-30 00:00:00,2,0.0,0.1,0.9,50,50,579,9,449,0.84,2,1.0,40,40
1099,mike,08012017_SOM45,SOM45,2017-08-01 00:00:00,2,0.0,0.9,0.1,50,50,479,6,350,0.78,2,1.0,40,40


In [5]:
r.shape

(97, 18)

## load in csv files (from running exportTrials.m) added in 'Laser Given' column in trials for new data format following opto changes

In [6]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given', 'Laser Given']

root_dir = '/Users/Sabatini Lab/GitHub/mouse_bandit/data/mike_data/trial_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [7]:
len(trial_df)

97

In [8]:
r.tail(10)

,Owner,Session ID,Mouse ID,Date,Phase,Laser Stim Prob,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
1070,mike,07252017_SOM41,SOM41,2017-07-25 00:00:00,2,0.0,0.1,0.9,50,50,432,6,344,0.89,2,1.0,40,40
1078,mike,07262017_SOM44,SOM44,2017-07-26 00:00:00,2,0.0,0.9,0.1,50,50,301,4,234,0.85,2,1.0,40,40
1079,mike,07262017_SOM45,SOM45,2017-07-26 00:00:00,2,0.0,0.9,0.1,50,50,548,8,433,0.86,2,1.0,40,40
1082,mike,07272017_SOM41,SOM41,2017-07-27 00:00:00,2,0.0,0.1,0.9,50,50,315,5,250,0.86,2,1.0,40,40
1084,mike,07272017_SOM44,SOM44,2017-07-27 00:00:00,2,0.0,0.1,0.9,50,50,222,3,162,0.80,2,1.0,40,40
1085,mike,07272017_SOM45,SOM45,2017-07-27 00:00:00,2,0.0,0.9,0.1,50,50,578,8,443,0.86,2,1.0,40,40
1088,mike,07282017_SOM41,SOM41,2017-07-28 00:00:00,2,0.0,0.9,0.1,50,50,87,1,61,0.74,2,1.0,40,40
1093,mike,07302017_SOM41,SOM41,2017-07-30 00:00:00,2,0.0,0.9,0.1,50,50,545,8,445,0.90,2,1.0,40,40
1095,mike,07302017_SOM45,SOM45,2017-07-30 00:00:00,2,0.0,0.1,0.9,50,50,579,9,449,0.84,2,1.0,40,40
1099,mike,08012017_SOM45,SOM45,2017-08-01 00:00:00,2,0.0,0.9,0.1,50,50,479,6,350,0.78,2,1.0,40,40


## convert into 1 feature matrix

In [9]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [10]:
master_matrix.shape

(40306, 53)

In [11]:
master_matrix['Higher p port'].mean()

0.84496104798293059

## Stephenson et al acquired ~6500 trials per animal

In [12]:
master_matrix['Session ID'].unique()

array(['06142017_SOM42', '06152017_SOM42', '06162017_SOM42',
       '06192017_SOM42', '06192017_SOM45', '06202017_SOM42',
       '06202017_SOM44', '06212017_SOM42', '06212017_SOM45',
       '06222017_SOM42', '06222017_SOM44', '06222017_SOM45',
       '06232017_SOM42', '06232017_SOM44', '06232017_SOM45',
       '06262017_PV30', '06262017_SOM41', '06262017_SOM42',
       '06262017_SOM44', '06262017_SOM45', '06272017_PV30',
       '06272017_SOM41', '06272017_SOM42', '06272017_SOM44',
       '06272017_SOM45', '06282017_PV30', '06282017_SOM41',
       '06282017_SOM42', '06282017_SOM44', '06282017_SOM45',
       '06292017_PV30', '06292017_SOM41', '06292017_SOM42',
       '06292017_SOM44', '06292017_SOM45', '06302017_PV30',
       '06302017_SOM41', '06302017_SOM42', '06302017_SOM44',
       '06302017_SOM45', '07032017_PV30', '07032017_SOM42',
       '07032017_SOM44', '07032017_SOM45', '07042017_PV30',
       '07042017_SOM41', '07042017_SOM42', '07042017_SOM44',
       '07052017_PV30', '070520

In [13]:
master_matrix.tail(10)

,Mouse ID,Session ID,Trial,Block Trial,Block Reward,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,...,1_Port,1_Reward,1_ITI,1_trialDuration,0_ITI,0_trialDuration,Decision,Switch,Higher p port,Reward
459,SOM45,08012017_SOM45,470.0,75.0,41.0,2.0,-2.0,0.622,1.0,0.0,...,0.506,1.0,0.0,11.488,0.593,1.480,0.0,1.0,1.0,1.0
460,SOM45,08012017_SOM45,471.0,76.0,42.0,1.0,1.0,0.347,1.0,0.0,...,0.593,0.0,1.0,1.480,0.622,3.372,0.0,0.0,1.0,1.0
461,SOM45,08012017_SOM45,472.0,77.0,43.0,2.0,2.0,0.527,0.0,1.0,...,0.622,0.0,1.0,3.372,0.347,1.391,1.0,1.0,0.0,0.0
462,SOM45,08012017_SOM45,473.0,78.0,43.0,1.0,-1.0,0.518,0.0,1.0,...,0.347,1.0,0.0,1.391,0.527,1.262,0.0,1.0,1.0,1.0
463,SOM45,08012017_SOM45,474.0,79.0,44.0,1.0,1.0,0.338,0.0,1.0,...,0.527,0.0,1.0,1.262,0.518,1.877,0.0,0.0,1.0,1.0
464,SOM45,08012017_SOM45,475.0,80.0,45.0,2.0,2.0,0.326,0.0,1.0,...,0.518,0.0,1.0,1.877,0.338,5.229,0.0,0.0,1.0,1.0
465,SOM45,08012017_SOM45,476.0,81.0,46.0,3.0,3.0,0.509,0.0,0.0,...,0.338,0.0,1.0,5.229,0.326,2.689,0.0,0.0,1.0,1.0
466,SOM45,08012017_SOM45,477.0,82.0,47.0,4.0,4.0,0.564,0.0,1.0,...,0.326,0.0,1.0,2.689,0.509,3.930,0.0,0.0,1.0,1.0
467,SOM45,08012017_SOM45,478.0,83.0,48.0,5.0,5.0,0.543,1.0,0.0,...,0.509,0.0,1.0,3.930,0.564,2.417,0.0,0.0,1.0,1.0
468,SOM45,08012017_SOM45,479.0,84.0,49.0,6.0,6.0,0.371,1.0,0.0,...,0.564,0.0,1.0,2.417,0.543,1.329,0.0,0.0,1.0,1.0


## Save combined feature matrix  

In [14]:
master_matrix.to_csv(os.path.join(root_dir,'all_9010_high.csv'))

In [124]:
#master_matrix.to_csv(os.path.join(root_dir,'all_8020_high.csv'))